In [1]:
import pandas as pd
import re
import string
import unicodedata

In [2]:
special_case = {'dđ': 'đ'}
not_in_vietnamese = ['w', 'f', 'j', 'z', 'W', 'F', 'J', 'Z']

INVALID = 'aaaaa'

def remove_diacritics(text: str) -> str:
    nfkd = unicodedata.normalize('NFD', text)
    return ''.join(ch for ch in nfkd if unicodedata.category(ch) != 'Mn')

def replace_word(sentence: str, old: str, new: str) -> str:
    words = sentence.split()
    replaced = [new if w == old else w for w in words]
    return " ".join(replaced)

def modify_word(w):
    """
    Sửa lỗi các từ như "dđ" và các ký tự lặp lại (ví dụ: "haaa" thành "ha").
    Hàm này chỉ xử lý phần chữ cái của từ.
    """
    if len(w) == 0:
        return w
    
    # Xử lý các ký tự lặp lại
    if w[0].islower():
        for c in string.ascii_lowercase:
            while c + c in w:
                w = w.replace(c + c, c)
        for c in not_in_vietnamese:
            w = w.replace(c, '')
    
    # Xử lý các trường hợp đặc biệt
    for old, new in special_case.items():
        if old in w:
            w = w.replace(old, new)
            
    return w

def fix_sentence(s):
    """
    Sửa lỗi từng từ trong một câu trong khi vẫn giữ lại dấu câu.
    """
    # Tìm tất cả các từ (chuỗi chữ cái) và áp dụng hàm modify_word cho chúng
    # Mọi thứ khác (dấu câu, khoảng trắng) sẽ được giữ nguyên
    return re.sub(r'[a-zA-ZÀ-ỹ]+', lambda m: modify_word(m.group(0)), s)

In [3]:
df = pd.read_csv('./data/vihallu-public-test.csv',encoding='utf-8')
df['prompt'] = df['prompt'].apply(fix_sentence)

In [4]:
for i, (c, p, r) in enumerate(zip(df['context'], df['prompt'], df['response'])):
    p = p.replace('Adversarial Question: ', '').strip()

    new_response = remove_diacritics(r)
    response_dicts = {}

    for w, new_w in zip(r.split(' '), new_response.split(' ')):
        if response_dicts.get(new_w) is None:
            response_dicts[new_w] = w
        elif response_dicts[new_w] != w:
            response_dicts[new_w] = INVALID

    new_context = remove_diacritics(c)
    response_dicts = {}

    for w, new_w in zip(c.split(' '), new_context.split(' ')):
        if response_dicts.get(new_w) is None:
            response_dicts[new_w] = w
        elif response_dicts[new_w] != w:
            response_dicts[new_w] = INVALID

    new_prompt = remove_diacritics(p)
    
    for w, new_w in zip(p.split(' '), new_prompt.split(' ')):
        if response_dicts.get(new_w) is not None and response_dicts.get(new_w) != INVALID:

            new_prompt = replace_word(new_prompt, new_w, response_dicts.get(new_w))
        else:
            new_prompt = replace_word(new_prompt, new_w, w)
    df.loc[i, 'prompt'] = new_prompt


In [ ]:
df.to_csv('./data/vihallu-public-test-2.csv', index=False, encoding='utf-8')

In [6]:
df.head(40)

,id,context,prompt,response,predict_label
0,b709059b-b3b6-4ac2-bb88-2c794e2cc219,"Putin ngày 14 tháng 10 năm 2009, đưa ra đề ngh...",Ý nhĩa của viẹc tổ chức cuộc thi Intervision l...,"Cuộc thi Intervision, dự kiến lấy cảm hứng từ ...",NaN
1,7dc35ef5-c4b7-4538-ab90-627b9cbd896e,"Thông qua những quyết nghị này, Viện nguyên lã...",Quyền lực của Viện nguyên lão đã giám mạnh khi...,"Sai. Thực tế, Viện nguyên lão vẫn duy trì quyề...",NaN
2,cfdfa010-f61c-4845-91c9-23f79be2b88b,Nơi cư ngụ truyền thống Mông Cổ được gọi là mộ...,Theo quan điểm của ai thì kiến trúc truyền thố...,"Ngoài N. Chultem, một số học giả như nhà sử họ...",NaN
3,31b33c97-2f59-4e72-8707-f47de204d7f9,Là thành phố thủ đô và có vị trí ở khu vực tru...,Hà Nội hiện này có tuyến đường sắt quốc tế kết...,Hà Nội hiện nay có tuyến đường sắt quốc tế trự...,NaN
4,a2c83a00-e8b7-4236-86ce-5e0104df074a,Từ 1974 đến nay tốc độ phát triển tuy chậm lại...,Câu hỏi gài bẫy: Tiền đề đề trợ thành một quốc...,Tiền đề để Nhật Bản trở thành quốc gia cho vay...,NaN
5,c5972f72-948c-4907-a0d9-5a5077d263eb,"Vào năm 1580, các tu sĩ dòng Tên thiết lập Mai...",Các nhà thần học và khoa học muốn tìm đến S.J ...,Các nhà thần học và khoa học muốn tìm đến S.J ...,NaN
6,c9cb5df4-6c6a-44db-977f-b5a9d59926f0,"Hiện nay, do đời sống được nâng cao hơn, cơ cấ...","Các gia đình, nhà hàng hiện nay trình bày chén...",Các gia đình và nhà hàng hiện nay thường san r...,NaN
7,87ae8fca-aa1d-4ba4-bebe-fed2c811dd29,Lãnh thổ Thành Vatican là bộ phận của Mons Vat...,"Trước năm 1929, khu vực lãnh thổ Thành Vatican...","Trước năm 1929, khu vực lãnh thổ Thành Vatican...",NaN
8,faa318fc-0679-486c-98fd-54823a2374f9,"Vì là vị hoàng đế đầu tiên, Augustus đã đảm nh...","Mặc dù Augustus đã tự gọi mình là ""Đệ nhất côn...",Augustus đã ngay lập tức xóa bỏ các thể chế cộ...,NaN
9,1d0a69e2-146d-492a-8c4a-3f77b7395c6a,"Ngày 7/9/1945, tờ Báo Tranh đấu, cơ quan ngôn ...",Ý nghĩa của thông cáo do Trần Văn Giàu đưa ra ...,Thông cáo của Trần Văn Giàu không chỉ kêu gọi ...,NaN


In [ ]:
# #In từ điểm ra xem vui vui th
# s = set()

# prompt = df['prompt']
# for p in prompt:
#     #tach cac tu cua p ra roi dua vao set
#     words = p.split(' ')
#     for w in words:
#         s.add(w)

# print(len(s))

# dicts = {}
# for w in s:
#     dicts[w] = 0

# for p in prompt:
#     words = p.split(' ')
    
#     for w in words:
#         if w in dicts:
#             dicts[w] += 1

# dicts = dict(sorted(dicts.items(), key=lambda item: item[1], reverse=True))

# with open('vocab.csv', 'w', encoding='utf-8') as f:
#     for w in dicts:
#         f.write(f'{w},{dicts[w]}\n')

12887
